In [14]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import concatenate
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import glob
import cv2
import os
import locale
import os

In [15]:
file = "epsilon-3"

### Load in `f` data

In [17]:
df = pd.read_csv(file + "-f.csv", sep=",")
df.T

,0,1,2,3,4,5,6,7,8,9,...,12663,12664,12665,12666,12667,12668,12669,12670,12671,12672
current_player,1.000000,0.000000,1.000000,0.000000,1.00000,0.000000,1.000000,0.000000,1.000000,0.000000,...,1.0,0.000000,1.0,0.0,1.00000,0.0,1.0,0.000000,1.000000,0.00000
is_current_player_hand_winner,1.000000,0.000000,1.000000,0.000000,1.00000,0.000000,1.000000,0.000000,1.000000,0.000000,...,1.0,0.000000,1.0,0.0,1.00000,0.0,1.0,1.000000,0.000000,1.00000
is_current_player_game_winner,1.000000,0.000000,1.000000,0.000000,1.00000,0.000000,1.000000,0.000000,1.000000,0.000000,...,0.0,1.000000,0.0,1.0,0.00000,1.0,0.0,1.000000,0.000000,1.00000
current_player_end_hand_score_advantage,6.000000,-6.000000,6.000000,-6.000000,6.00000,-6.000000,6.000000,-6.000000,6.000000,-6.000000,...,11.0,-11.000000,11.0,-11.0,11.00000,-11.0,11.0,45.000000,-45.000000,45.00000
current_player_score,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,81.0,98.000000,81.0,98.0,81.00000,98.0,81.0,98.000000,92.000000,98.00000
opponent_score,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,98.0,81.000000,98.0,81.0,98.00000,81.0,98.0,92.000000,98.000000,92.00000
current_player_deadwood,41.000000,67.000000,36.000000,64.000000,30.00000,38.000000,30.000000,38.000000,11.000000,17.000000,...,23.0,17.000000,11.0,17.0,11.00000,17.0,6.0,15.000000,20.000000,0.00000
current_player_num_hit_cards,2.000000,5.000000,4.000000,7.000000,4.00000,5.000000,4.000000,5.000000,1.000000,2.000000,...,4.0,9.000000,3.0,9.0,3.00000,9.0,2.0,2.000000,2.000000,4.00000
num_melds,1.000000,0.000000,1.000000,0.000000,1.00000,1.000000,1.000000,1.000000,1.000000,2.000000,...,1.0,1.000000,2.0,1.0,2.00000,1.0,2.0,2.000000,2.000000,3.00000
point_sum_melds,3.000000,0.000000,3.000000,0.000000,3.00000,24.000000,3.000000,24.000000,15.000000,42.000000,...,29.0,27.000000,35.0,27.0,35.00000,27.0,45.0,32.000000,54.000000,44.00000


In [18]:
f = [
#      'current_player',
#      'is_current_player_hand_winner',
#      'is_current_player_game_winner',
     'current_player_end_hand_score_advantage',
#      'current_player_score',
#      'opponent_score',
#      'current_player_deadwood',
     'current_player_num_hit_cards',
     'num_melds',
#      'point_sum_melds',
     'num_combos',
#      'point_sum_combos',
#      'num_knock_cache',
#      'point_sum_knock_cache',
#      'num_load_cards',
#      'point_sum_load_cards',
     'turns_taken',
#      'num_nearby_opponent_cards',
#      'discard_danger'
]


x = f[1:]
y = "current_player_end_hand_score_advantage"

print("x =", x)
print("y =", y)

x = ['current_player_num_hit_cards', 'num_melds', 'num_combos', 'turns_taken']
y = current_player_end_hand_score_advantage


In [19]:
features = df[f]

### Load in `m` data

In [21]:
card_matrices = []

with open(file + '-c.csv') as f:
    for line in f.readlines():
        card_matrices.append(eval(line))

card_matrices = np.array(card_matrices)
card_matrices

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 1, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        ...,
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0]],

       [[1, 0, 0, ..., 1, 0, 0],
        [0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

### split train and test data

In [22]:
trainFeaturesX, testFeaturesX, trainMatricesX, testMatricesX = train_test_split(df[x], card_matrices, test_size=0.25, random_state=42)
trainY, testY = train_test_split(df[[y]], test_size=0.25, random_state=42)

display(trainFeaturesX.T)
display(trainY.T)

,6109,8380,12356,11715,1488,9229,4919,8456,7037,4047,...,4426,466,6265,5734,11284,11964,5191,5390,860,7270
current_player_num_hit_cards,2.0,4.0,3.0,2.0,5.0,2.0,7.0,3.0,2.0,9.0,...,8.0,2.0,4.0,3.0,3.0,5.0,6.0,5.0,6.0,6.0
num_melds,2.0,0.0,1.0,2.0,1.0,2.0,0.0,1.0,2.0,0.0,...,0.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0
num_combos,0.0,2.0,1.0,0.0,3.0,1.0,4.0,1.0,0.0,6.0,...,4.0,0.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,3.0
turns_taken,3.0,2.0,1.0,6.0,20.0,10.0,1.0,1.0,4.0,16.0,...,6.0,13.0,1.0,21.0,4.0,1.0,10.0,10.0,1.0,7.0


,6109,8380,12356,11715,1488,9229,4919,8456,7037,4047,...,4426,466,6265,5734,11284,11964,5191,5390,860,7270
current_player_end_hand_score_advantage,-18.0,-8.0,11.0,9.0,-3.0,15.0,-12.0,4.0,33.0,1.0,...,-16.0,5.0,25.0,-17.0,-14.0,-20.0,5.0,13.0,-21.0,12.0


### create the MLP model

In [23]:
mlp = Sequential()
mlp.add(Dense(8, input_dim=trainFeaturesX.shape[1], activation="relu"))
mlp.add(Dense(4, activation="relu"))
if False:
    mlp.add(Dense(1, activation = "linear"))

### Create CNN model

In [24]:
height = len(card_matrices[0])
width = len(card_matrices[0][0])
depth = 1
filters = (16, 32, 64)

inputShape = (height, width, depth)
chanDim = -1

inputs = Input(shape=inputShape)

for (i, f) in enumerate(filters):
    # if this is the first CONV layer then set the input appropriately
    if i == 0:
        x = inputs

    # CONV => RELU => BN => POOL
    x = Conv2D(f, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

# flatten the volume, then FC => RELU => BN => DROPOUT
x = Flatten()(x)
x = Dense(16)(x)
x = Activation("relu")(x)
x = BatchNormalization(axis=chanDim)(x)
x = Dropout(0.5)(x)

# match the number of nodes coming out of the MLP
x = Dense(4)(x)
x = Activation("relu")(x)

if False:
    x = Dense(1, activation="linear")(x)

cnn = Model(inputs, x)

### Combine

In [25]:
combinedInput = concatenate([mlp.output, cnn.output])

x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)

model = Model(inputs=[mlp.input, cnn.input], outputs=x)

### Compile

In [26]:
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

### Train

In [27]:
model.fit(
    [trainFeaturesX, trainMatricesX], trainY,
    validation_data=([testFeaturesX, testMatricesX], testY),
    epochs=200, batch_size=8)

Epoch 1/200
1188/1188 [==============================] - 19s 16ms/step - loss: 99.2336 - val_loss: 98.8134
Epoch 2/200
1188/1188 [==============================] - 18s 15ms/step - loss: 98.7446 - val_loss: 98.2429
Epoch 3/200
1188/1188 [==============================] - 18s 15ms/step - loss: 98.0047 - val_loss: 97.9856
Epoch 4/200
1188/1188 [==============================] - 18s 15ms/step - loss: 96.8228 - val_loss: 96.9122
Epoch 5/200
1188/1188 [==============================] - 18s 15ms/step - loss: 95.1359 - val_loss: 95.9604
Epoch 6/200
1188/1188 [==============================] - 19s 16ms/step - loss: 93.7638 - val_loss: 93.7194
Epoch 7/200
1188/1188 [==============================] - 18s 15ms/step - loss: 92.8481 - val_loss: 92.9223
Epoch 8/200
1188/1188 [==============================] - 19s 16ms/step - loss: 92.0466 - val_loss: 94.1657
Epoch 9/200
1188/1188 [==============================] - 17s 15ms/step - loss: 91.1999 - val_loss: 90.6336
Epoch 10/200
1188/1188 [=============

Epoch 77/200
1188/1188 [==============================] - 13s 11ms/step - loss: 61.9732 - val_loss: 72.7137
Epoch 78/200
1188/1188 [==============================] - 11s 10ms/step - loss: 62.2345 - val_loss: 72.9341
Epoch 79/200
1188/1188 [==============================] - 8s 7ms/step - loss: 61.8316 - val_loss: 72.8281
Epoch 80/200
1188/1188 [==============================] - 8s 7ms/step - loss: 61.9350 - val_loss: 73.3173
Epoch 81/200
1188/1188 [==============================] - 9s 8ms/step - loss: 62.3083 - val_loss: 71.7544
Epoch 82/200
1188/1188 [==============================] - 9s 8ms/step - loss: 61.8661 - val_loss: 73.2116
Epoch 83/200
1188/1188 [==============================] - 9s 7ms/step - loss: 61.5917 - val_loss: 72.0266
Epoch 84/200
1188/1188 [==============================] - 8s 7ms/step - loss: 60.9663 - val_loss: 73.4931
Epoch 85/200
1188/1188 [==============================] - 11s 9ms/step - loss: 60.9597 - val_loss: 72.0356
Epoch 86/200
1188/1188 [=================

1188/1188 [==============================] - 8s 7ms/step - loss: 54.4501 - val_loss: 69.1710
Epoch 154/200
1188/1188 [==============================] - 8s 7ms/step - loss: 54.8582 - val_loss: 69.8927
Epoch 155/200
1188/1188 [==============================] - 8s 7ms/step - loss: 54.5761 - val_loss: 68.6793
Epoch 156/200
1188/1188 [==============================] - 8s 7ms/step - loss: 54.8251 - val_loss: 68.8995
Epoch 157/200
1188/1188 [==============================] - 8s 7ms/step - loss: 53.7955 - val_loss: 68.6868
Epoch 158/200
1188/1188 [==============================] - 8s 7ms/step - loss: 54.3301 - val_loss: 69.0016
Epoch 159/200
1188/1188 [==============================] - 8s 7ms/step - loss: 53.8900 - val_loss: 69.4459
Epoch 160/200
1188/1188 [==============================] - 8s 7ms/step - loss: 54.2011 - val_loss: 71.8447
Epoch 161/200
1188/1188 [==============================] - 8s 7ms/step - loss: 54.9224 - val_loss: 69.9180
Epoch 162/200
1188/1188 [==========================

### Testing

In [28]:
preds = model.predict([testFeaturesX, testMatricesX])
diff = preds.T[0] - testY.values.T[0]

print("average difference =", np.average(diff))

average difference = -0.3390417997112557


In [29]:
print(model.predict([testFeaturesX, testMatricesX]).T[0][:10])

[ 7.0036936  8.719218   3.8741577 11.872931  15.929278  -9.621895
  9.8492155  9.234694   3.2002532  5.3558483]


In [30]:
print(testY.values.T[0][:10])

[17. 15. 16.  8. 11. 22. 19. 13.  3. 12.]


In [32]:
model.save('best_model_2.h5')